In [12]:
import tensorflow as tf
import numpy as np

In [73]:
n_inputs = 28*28
learning_rate = 0.2

In [74]:
X = tf.placeholder(tf.float32, (None,n_inputs), name = "X")
y = tf.placeholder(tf.int64,(None),name = "y")

In [75]:
X.get_shape()[1]

Dimension(784)

In [76]:
def neuron_layer(X,n_neurons,name,activation=None):
    with tf.name_scope("name"):
        #인풋 행렬의 행을 가져온다.
        x_inputs = int(X.get_shape()[1])
        #절단 가우시안 분포를 쓰기 위해, 2/root(n_neurons + n_inputs)꼴의 표준편차를 정의한다.
        std_dev = 2/np.sqrt(n_neurons + x_inputs)
        #절단 가우시안 분포로 가중치 행렬을 초기화한다.
        init = tf.truncated_normal((x_inputs,n_neurons),stddev=std_dev)
        w = tf.Variable(init,name="kernel")
        #0으로 이루어진 편향 항을 추가한다. 그 차원은 1 X (열의 수)가 된다. 이렇게 되면, (샘플 수) X (열의 수)인 행렬에 (열의 수)만큼 
        #반복해서 곱을 해준다.
        b = tf.Variable(tf.zeros([n_neurons]),name="bias")
        #행렬곱으로 묶는다.
        z = tf.matmul(X,w)+b
        #활성화 함수가 지정되었다면, 해당 활성화 함수로 변환된 값을 출력한다.
        if activation is not None:
            return activation(z)
        else:
            return z

In [77]:
#모델을 구성한다.

with tf.name_scope("model"):
    hidden1 = neuron_layer(X,300,"hidden1",tf.nn.relu)
    hidden2 = neuron_layer(hidden1,100,"hidden2",tf.nn.relu)
    logits = neuron_layer(hidden2,10,"output")

In [78]:
#목적함수를 정의한다. 타겟값은 1을 곱하고, 나머지 값은 0을 곱하는 크로스 엔트로피를 목적함수로 정의한다.
with tf.name_scope("optimizer"):
    #이 함수는 출력된 로짓값에 소프트맥스를 적용하고, 소프트맥스로 변환된 확률값을 cross entropy에 집어넣어 해당 훈련의 엔트로피 값을 
    #계산해준다.
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    #이 때, 이 크로스 엔트로피를 전부 더하고, 평균을 낸 결과를 최소화 하는것이 목적이 된다.
    loss = tf.reduce_mean(xentropy,name="loss")

In [79]:
#목적함수를 각 가중치로 미분한 그레디언트 벡터를, 각 가중치에서 계속 빼서 업데이트 하는 경사하강법을 적용한다.

with tf.name_scope("gradient"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #경사하강법 최적화를 optimizer로 할당한 다음, optimizer를 최소화 방법(minimize)로 목적함수를 최적화시킨다.
    training_op = optimizer.minimize(loss)

In [82]:
# 정확도 평가용 그래프를 정의한다.
with tf.name_scope("eval"):
    # top_k함수는 어떤 출력값이 비교값(타겟값)과 비교했을 때 가장 높은 출력값을 보인 타겟값을 True로, 나머지는 False인 텐서를 반환한다.
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [ ]:
init = tf.global_variable_initialzier()

with tf.Session() as sess:
    init.run()
    
    